<a href="https://colab.research.google.com/github/pachterlab/scATAK/blob/main/scATAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/data/min-R1.fastq.gz?raw=true > min-R1.fastq.gz
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/data/min-R2.fastq.gz?raw=true > min-R2.fastq.gz
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/data/min-R3.fastq.gz?raw=true > min-R3.fastq.gz
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/lib/737K-cratac-v1.txt?raw=true > 737K-cratac-v1.txt

In [2]:
# install minimap for mapping paired end reads
!wget --quiet --show-progress "https://github.com/lh3/minimap2/releases/download/v2.24/minimap2-2.24_x64-linux.tar.bz2"
!tar -jxvf minimap2-2.24_x64-linux.tar.bz2
!cp minimap2-2.24_x64-linux/minimap2 /usr/local/bin

# Insstall sambamba for sam file manipulation
!wget --quiet --show-progress https://github.com/biod/sambamba/releases/download/v0.8.2/sambamba-0.8.2-linux-amd64-static.gz
!gunzip sambamba-0.8.2-linux-amd64-static.gz
!chmod +x /content/sambamba-0.8.2-linux-amd64-static
!mv sambamba-0.8.2-linux-amd64-static /usr/local/bin/sambamba

# Genrich for peak callling
!git clone https://github.com/jsh58/Genrich.git
!cd Genrich && make && cp Genrich /usr/local/bin

# bedtools for slicing the peaks
!wget --quiet --show-progress https://github.com/arq5x/bedtools2/releases/download/v2.30.0/bedtools.static.binary
!chmod +x /content/bedtools.static.binary
!mv /content/bedtools.static.binary /usr/local/bin/bedtools

# kb python for quantification
!pip install --quiet kb-python

minimap2-2.24_x64-l 100%[===================>]   2.04M  --.-KB/s    in 0.06s   
minimap2-2.24_x64-linux/
minimap2-2.24_x64-linux/LICENSE.txt
minimap2-2.24_x64-linux/k8
minimap2-2.24_x64-linux/README-js.md
minimap2-2.24_x64-linux/paftools.js
minimap2-2.24_x64-linux/minimap2
minimap2-2.24_x64-linux/README.md
minimap2-2.24_x64-linux/cookbook.md
minimap2-2.24_x64-linux/minimap2.1
minimap2-2.24_x64-linux/NEWS.md
sambamba-0.8.2-linu 100%[===================>]   4.73M  --.-KB/s    in 0.07s   
Cloning into 'Genrich'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 532 (delta 5), reused 11 (delta 5), pack-reused 521
Receiving objects: 100% (532/532), 445.96 KiB | 8.11 MiB/s, done.
Resolving deltas: 100% (302/302), done.
gcc -g -Wall -std=gnu99 -O2 -o Genrich Genrich.c -lz -lm
bedtools.static.bin 100%[===================>]  40.05M  76.4MB/s    in 0.5s    


In [3]:
!minimap2

Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]
Options:
  Indexing:
    -H           use homopolymer-compressed k-mer (preferrable for PacBio)
    -k INT       k-mer size (no larger than 28) [15]
    -w INT       minimizer window size [10]
    -I NUM       split index for every ~NUM input bases [4G]
    -d FILE      dump index to FILE []
  Mapping:
    -f FLOAT     filter out top FLOAT fraction of repetitive minimizers [0.0002]
    -g NUM       stop chain enlongation if there are no minimizers in INT-bp [5000]
    -G NUM       max intron length (effective with -xsplice; changing -r) [200k]
    -F NUM       max fragment length (effective with -xsr or in the fragment mode) [800]
    -r NUM[,NUM] chaining/alignment bandwidth and long-join bandwidth [500,20000]
    -n INT       minimal number of minimizers on a chain [3]
    -m INT       minimal chaining score (matching bases minus log gap penalty) [40]
    -X           skip self and dual mappings (for the all-vs-all m

In [4]:
!sambamba


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)


Usage: sambamba [command] [args...]

  Available commands:

    view        view contents and convert from one format
                to another (SAM/BAM/JSON/UNPACK)
    index       build index (BAI)
    merge       merge files (BAM)
    sort        sort file (BAM)
    slice       slice file (BAM using BED)
    markdup     mark or remove duplicates (BAM)
    subsample   subsample (BAM)
    flagstat    output statistics (BAM)
    depth       output statistics (BAM)
    validate    simple validator (BAM)

  No longer recommended:

    mpileup     parallel execution of samtools (BAM)

To get help on a particular command, call it without args.

Global options

    -q          quiet mode (do not show banner)

For bug reports and feature requests see

       https://github.com/biod/



In [5]:
!Genrich

Error! Need input/output files
Usage: ./Genrich  -t <file>  -o <file>  [optional arguments]
Required arguments:
  -t  <file>       Input SAM/BAM file(s) for experimental sample(s)
  -o  <file>       Output peak file (in ENCODE narrowPeak format)
Optional I/O arguments:
  -c  <file>       Input SAM/BAM file(s) for control sample(s)
  -f  <file>       Output bedgraph-ish file for p/q values
  -k  <file>       Output bedgraph-ish file for pileups and p-values
  -b  <file>       Output BED file for reads/fragments/intervals
  -R  <file>       Output file for PCR duplicates (only with -r)
Filtering options:
  -r               Remove PCR duplicates
  -e  <arg>        Comma-separated list of chromosomes to exclude
  -E  <file>       Input BED file(s) of genomic regions to exclude
  -m  <int>        Minimum MAPQ to keep an alignment (def. 0)
  -s  <float>      Keep sec alns with AS >= bestAS - <float> (def. 0)
  -y               Keep unpaired alignments (def. false)
  -w  <int>        Keep unp

In [6]:
!bedtools

bedtools is a powerful toolset for genome arithmetic.

Version:   v2.30.0
About:     developed in the quinlanlab.org and by many contributors worldwide.
Docs:      http://bedtools.readthedocs.io/
Code:      https://github.com/arq5x/bedtools2
Mail:      https://groups.google.com/forum/#!forum/bedtools-discuss

Usage:     bedtools <subcommand> [options]

The bedtools sub-commands include:

[ Genome arithmetic ]
    intersect     Find overlapping intervals in various ways.
    window        Find overlapping intervals within a window around an interval.
    closest       Find the closest, potentially non-overlapping interval.
    coverage      Compute the coverage over defined intervals.
    map           Apply a function to a column for each overlapping interval.
    genomecov     Compute the coverage over an entire genome.
    merge         Combine overlapping/nearby intervals into a single interval.
    cluster       Cluster (but don't merge) overlapping/nearby intervals.
    complement

In [7]:
!wget --quiet --show-progress "http://ftp.ensembl.org/pub/release-104/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz"

Homo_sapiens.GRCh38 100%[===================>] 840.39M  31.7MB/s    in 27s     


In [8]:
!time minimap2 \
-d ref.mmi \
Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

tcmalloc: large alloc 1073741824 bytes == 0x69d5e000 @  0x7fc7311352a4 0x414d2d 0x406c48 0x4129cf 0x7fc73092e6db 0x7fc73065761f
tcmalloc: large alloc 2147483648 bytes == 0x27d0e4000 @  0x7fc7311352a4 0x406fee 0x4129cf 0x7fc73092e6db 0x7fc73065761f
[M::mm_idx_gen::110.853*1.64] collected minimizers
[M::mm_idx_gen::144.313*1.72] sorted minimizers
[M::main::174.008*1.50] loaded/built the index for 194 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 194
[M::mm_idx_stat::175.288*1.49] distinct minimizers: 100159079 (38.79% are singletons); average occurrences: 5.540; average spacing: 5.586; total length: 3099750718
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -d ref.mmi Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
[M::main] Real time: 177.054 sec; CPU: 263.420 sec; Peak RSS: 11.290 GB

real	2m57.672s
user	3m53.645s
sys	0m30.384s


In [9]:
!time minimap2 -ax sr -t2 ref.mmi \
   <(paste \
     <(zcat min-R1.fastq.gz) \
     <(zcat min-R2.fastq.gz) \
     | awk -v blen="16" '{if(NR%4==1) header=$1; if(NR%4==2) print header"_"substr($2, 0, blen)"_\n"$1; if(NR%4==3 || NR%4==0) print $1;}') \
   <(paste \
     <(zcat min-R3.fastq.gz) \
     <(zcat min-R2.fastq.gz) \
     | awk -v blen="16" '{if(NR%4==1) header=$1; if(NR%4==2) print header"_"substr($2, 0, blen)"_\n"$1; if(NR%4==3 || NR%4==0) print $1;}') \
   > peaks.sam

tcmalloc: large alloc 1549877248 bytes == 0x17f764000 @  0x7f081fc431e7 0x40a494 0x40a9b8 0x402995 0x7f081f066c87 0x4018b9
[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::46.477*0.25] loaded/built the index for 194 target sequence(s)
[M::mm_mapopt_update::46.477*0.25] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 194
[M::mm_idx_stat::47.701*0.27] distinct minimizers: 100159079 (38.79% are singletons); average occurrences: 5.540; average spacing: 5.586; total length: 3099750718
[M::worker_pipeline::90.147*1.01] mapped 500000 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -ax sr -t2 ref.mmi /dev/fd/63 /dev/fd/62
[M::main] Real time: 91.150 sec; CPU: 91.765 sec; Peak RSS: 7.617 GB

real	1m31.670s
user	1m25.410s
sys	0m9.332s


In [10]:
!sambamba view -t 2 -f bam -S -o peaks.bam peaks.sam
!rm peaks.sam


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)



In [11]:
!time sambamba sort -n -t 2 -m 8GB --tmpdir=./ peaks.bam


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)


real	0m3.448s
user	0m6.375s
sys	0m0.227s


In [12]:
!time Genrich -t peaks.sorted.bam -o peaks.narrowPeak -f peaks.log -v

Processing experimental file #0: peaks.sorted.bam
  BAM records analyzed:      500166
    Unmapped:                 25581
    Supp./dups/lowQual:         166
    Paired alignments:       447022
    Unpaired alignments:      27397
  Fragments analyzed:        223511
    Full fragments:          223511
      (avg. length: 174.4bp)
- control file #0 not provided -
  Background pileup value: 0.012576
Peak-calling parameters:
  Genome length: 3099750718bp
  Significance threshold: -log(p) > 2.000
  Min. AUC: 200.000
  Max. gap between sites: 100bp
Peaks identified: 150592 (32552110bp)

real	0m20.514s
user	0m14.922s
sys	0m5.504s


In [14]:
!cat peaks.narrowPeak | bedtools sort | bedtools merge > atac_all_peaks.bed

In [23]:
!gunzip Homo_sapiens.GRCh38.dna.primary_assembly.fa \
&& fold -w 80 Homo_sapiens.GRCh38.dna.primary_assembly.fa > genome.fa

In [25]:
!bedtools getfasta -fi genome.fa -bed atac_all_peaks.bed -fo atac_all_peaks.fa

index file genome.fa.fai not found, generating...


In [31]:
!ls -lht atac_all_peaks.fa

-rw-r--r-- 1 root root 35M Sep 30 20:50 atac_all_peaks.fa


In [35]:
!time $(pip show kb-python | grep "Location" | cut -f2 -d":")/kb_python/bins/linux/kallisto/kallisto index \
-i peaks.idx \
atac_all_peaks.fa


[build] loading fasta file atac_all_peaks.fa
[build] k-mer length: 31
[build] warning: clipped off poly-A tail (longer than 10)
        from 2 target sequences
[build] warning: replaced 559 non-ACGUT characters in the input sequence
        with pseudorandom nucleotides
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 329802 contigs and contains 26747917 k-mers 



In [43]:
!cat atac_all_peaks.fa | awk '{if($1~/>/)print substr($1, 2)"\t"substr($1, 2)"\t"substr($1, 2)}' > t2g.txt

In [44]:
!head t2g.txt

1:10015-10460	1:10015-10460	1:10015-10460
1:11154-11266	1:11154-11266	1:11154-11266
1:29210-29373	1:29210-29373	1:29210-29373
1:29480-29798	1:29480-29798	1:29480-29798
1:99502-99868	1:99502-99868	1:99502-99868
1:104185-104255	1:104185-104255	1:104185-104255
1:143396-143517	1:143396-143517	1:143396-143517
1:149332-149519	1:149332-149519	1:149332-149519
1:180546-180929	1:180546-180929	1:180546-180929
1:181290-181834	1:181290-181834	1:181290-181834


In [45]:
!kb count \
-i peaks.idx \
-g t2g.txt \
-x 10xATAC \
-o out \
-w 737K-cratac-v1.txt \
-t 2 \
--h5ad \
min-R2.fastq.gz min-R1.fastq.gz min-R3.fastq.gz

usage: kb count [-h] [--tmp TMP] [--keep-tmp] [--verbose] -i INDEX -g T2G -x
                TECHNOLOGY [-o OUT] [-w WHITELIST] [-t THREADS] [-m MEMORY]
                [--strand {unstranded,forward,reverse}]
                [--workflow {standard,lamanno,nucleus,kite,kite:10xFB}] [--em]
                [--umi-gene] [--mm | --tcc] [--filter [{bustools}]]
                [--filter-threshold THRESH] [-c1 T2C] [-c2 T2C] [--overwrite]
                [--dry-run] [--loom | --h5ad] [--cellranger] [--gene-names]
                [--report] [--kallisto KALLISTO] [--bustools BUSTOOLS]
                [--parity {single,paired}] [--fragment-l L] [--fragment-s S]
                fastqs [fastqs ...]

Generate count matrices from a set of single-cell FASTQ files. Run `kb --list`
to view single-cell technology information.

positional arguments:
  fastqs                FASTQ files. For technology `SMARTSEQ`, all input
                        FASTQs are alphabetically sorted by path and paired in
      

In [46]:
!kb --list

List of supported single-cell technologies

Positions syntax: `input file index, start position, end position`
When start & end positions are None, refers to the entire file
Custom technologies may be defined by providing a kallisto-supported technology string
(see https://pachterlab.github.io/kallisto/manual)

name            description                            whitelist    barcode                    umi        cDNA                       
------------    -----------------------------------    ---------    -----------------------    -------    -----------------------    
10XV1           10x version 1                          yes          0,0,14                     1,0,10     2,None,None                
10XV2           10x version 2                          yes          0,0,16                     0,16,26    1,None,None                
10XV3           10x version 3                          yes          0,0,16                     0,16,28    1,None,None                
10XV3_ULTIMA    1